In [1]:
import pandas as pd

In [2]:
countries = pd.read_csv('countries.csv')
iso_table = pd.read_csv('ISO-3166.csv')

In [3]:
merged_df = pd.merge(iso_table, countries, left_on='name', right_on='country', how='right')
merged_df = merged_df.drop(columns=['name', 'alpha-3', 'country-code', 'iso_3166-2', 'region', 'sub-region', 'intermediate-region', 'region-code', 'sub-region-code', 'intermediate-region-code'])
iso = merged_df.rename(columns={'alpha-2': 'iso_code', 'country': 'country_name'})
iso.loc[iso['country_name'] == 'Namibia', 'iso_code'] = 'NA'

In [4]:
iso

,iso_code,country_name
0,AF,Afghanistan
1,AX,Åland Islands
2,AL,Albania
3,DZ,Algeria
4,AS,American Samoa
...,...,...
235,VI,Virgin Islands (U.S.)
236,WF,Wallis and Futuna
237,YE,Yemen
238,ZM,Zambia


In [5]:
airports = pd.read_csv('airports.csv')
airports.head()

,code,icao,name,latitude,longitude,elevation,url,time_zone,city_code,country,city,state,county,type
0,AAA,NTGA,Anaa,-17.350665,-145.511120,36,NaN,Pacific/Tahiti,AAA,PF,NaN,NaN,NaN,AP
1,AAB,YARY,Arrabury Airport,-26.696783,141.049092,328,NaN,Australia/Brisbane,AAB,AU,Tanbar,Queensland,Barcoo Shire,AP
2,AAC,HEAR,El Arish International Airport,31.074284,33.829172,85,NaN,Africa/Cairo,AAC,EG,Arish,Muhafazat Shamal Sina',NaN,AP
3,AAD,HCAD,Adado Airport,6.096286,46.637708,980,NaN,Africa/Khartoum,AAD,SO,Adado,NaN,NaN,AP
4,AAE,DABB,Les Salines Airport,36.821392,7.811857,36,NaN,Africa/Algiers,AAE,DZ,El Hadjar,Annaba,NaN,AP


In [6]:
duplicate_iata_count = airports['code'].duplicated().sum()
print(f"Number of duplicate IATA codes: {duplicate_iata_count}")
airports.__len__()

Number of duplicate IATA codes: 0


9770

In [7]:
merged_df = pd.merge(airports, iso, left_on='country', right_on='iso_code', how='inner')
merged_df.__len__()

9735

In [8]:
nan_country = merged_df[merged_df['country_name'].isnull()]
iso_codes = nan_country['country'].unique()
filtered_iso = iso_table[iso_table['alpha-2'].isin(iso_codes)]
filtered_iso_names = filtered_iso['name']
# Format each row for SQL entry
formatted_rows = []
for name in filtered_iso_names:
    formatted_row = f"('{name}')"
    formatted_rows.append(formatted_row)

# Join all formatted rows into a single string
formatted_text = ',\n'.join(formatted_rows)

# Export the formatted text to a plaintext file
with open('append_countries.txt', 'w') as file:
    file.write(formatted_text)

In [9]:
merged_df = merged_df.drop(columns=['icao', 'latitude', 'longitude', 'elevation', 'url', 'city_code', 'country', 'county', 'type'])
merged_df.head()

,code,name,time_zone,city,state,iso_code,country_name
0,AAA,Anaa,Pacific/Tahiti,NaN,NaN,PF,French Polynesia
1,AAB,Arrabury Airport,Australia/Brisbane,Tanbar,Queensland,AU,Australia
2,AAC,El Arish International Airport,Africa/Cairo,Arish,Muhafazat Shamal Sina',EG,Egypt
3,AAD,Adado Airport,Africa/Khartoum,Adado,NaN,SO,Somalia
4,AAE,Les Salines Airport,Africa/Algiers,El Hadjar,Annaba,DZ,Algeria


In [10]:
# Format each row for SQL entry
formatted_rows = []
for index, row in merged_df.iterrows():
    name = row['name'].replace("'", "\\'")
    if pd.isnull(row['city']):
        city = 'NULL'
    else: 
        city = row['city'].replace("'", "\\'")
        city = f"'{city}'"
    country_name = row['country_name'].replace("'", "\\'")
    formatted_row = f"INSERT INTO airport VALUES('{row['code']}', '{name}', {city}, '{country_name}')"
    formatted_rows.append(formatted_row)

# Join all formatted rows into a single string
formatted_text = ';\n'.join(formatted_rows)

# Export the formatted text to a plaintext file
with open('into_airport.txt', 'w') as file:
    file.write(formatted_text)

In [11]:
# Export the merged dataframe to a CSV file
merged_df.to_csv('airports_merged.csv', index=False)